In [5]:
import sys
sys.path.append("../../spc_utils/")

In [6]:
from spc_imports import *
set_up_plt()
import MDAnalysis as mda
import data_process
import glob

/home/sperez/bin/PYTHON_SCRIPTS/spc_imports.py:21: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  plt.rcParams["text.latex.preamble"] = [


In [51]:
%%time
skip=10
div=skip*2
for treant in b:
    atoms=['N','C','CA','CB']
    iterables=[np.arange(26,122),atoms]
    indices=pd.MultiIndex.from_product(iterables, names=['resid','atom'])
    print(treant)
    top= md.load(treant['traj_comp_a_pbc_protein.xtc'].abspath,top=treant['topo_protein.psf'].abspath)
    shifts=[]
    shifts2=[]
    for chain in ["A","B","C","D"]:
        trj= md.load(treant['traj_comp_a_pbc_protein.xtc'].abspath,top=treant['topo_protein.psf'].abspath
                     ,atom_indices=top.topology.select(f"segname PRO{chain} and not element H and not name H HA"))
        shifts0=[]
        n_frames=trj.n_frames
        step=n_frames//div
        for j in tqdm(range(0,n_frames-step,step), desc=f't({chain})',leave=True,smoothing=1):
            shifts0.append(md.nmr.chemical_shifts_spartaplus(trj[j:j+step:skip]))
#         if n_frames%div >skip :
#             shifts0.append(md.nmr.chemical_shifts_spartaplus(trj[j+step:n_frames:skip]))
        shifts0=pd.concat(shifts0,axis=1)
        shifts0=shifts0.reindex(indices)
        shifts0.columns=np.arange(0,len(shifts0.columns))
        shifts.append(pd.DataFrame({'theo':shifts0.mean(axis=1)}))
        shifts2.append(shifts0)
    df=pd.concat(shifts)
    df=pd.DataFrame(np.hstack([df.groupby(['resid','atom']).mean(),
                       df.groupby(['resid','atom']).sem()]),
             columns=['mean','sem'],
            index=df.groupby(['resid','atom']).mean().index)
    df2=pd.concat(shifts2)
    df2.to_pickle(treant['NMR/sparta_plus_raw.pkl'].abspath)
    my_list0=[]
    for atom in atoms:
        my_list0.append(df['mean'].loc[:,atom].to_numpy())
    for atom in atoms:
        my_list0.append(df['sem'].loc[:,atom].to_numpy())
    iterables=[['mean','sem'],atoms]
    columns=pd.MultiIndex.from_product(iterables, names=['var','atom'])
    df=pd.DataFrame(np.vstack(my_list0).T,index=df.index.levels[0],columns=columns)
    df.to_pickle(treant['NMR/sparta_plus.pkl'].abspath)

NameError: name 'b' is not defined

In [52]:
raw_data_dir = '../data/raw/'

Load the trajectory

In [53]:
state = '5VKH_3FB5_string'
pdbs = sorted(glob.glob(raw_data_dir+f'{state}/*gro'))
univ = mda.Universe(raw_data_dir+f'{state}/topo.psf',pdbs)

/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


Get the NMR shifts with ppm

In [71]:
%cd /home/sperez/Projects/nmr_assign_state/notebooks/

/data/sperez/Projects/nmr_assign_state/notebooks


In [111]:
def get_sparta_plus_chemical_shifts(univ,temp_dir='./',split_size=100,skip=1):
    import MDAnalysis as mda
    import mdtraj as md
    from tqdm import tqdm
    '''
    Use sparta_plus with the interface of mdtraj to get the chemical shifts of     the trajectory. The segids are concatenated as independent trajectories.

    Parameters
    ----------
    univ : Universe object that contains the protein.
    temp_dir : Temporary directory for the tmp trajectory.
    split_size : How to split the trajectory so the /tmp does not fill.
    skip : Amount of frames skiped.
      
    Returns
    -------
    df: Dataframe of chemical shifts (n_cs_nuclei,n_frames*n_segids)
    '''
    assert isinstance(temp_dir,str), f'{temp_dir} should be a str'
    assert isinstance(split_size,int), f'{split_size} should be a int'
    assert isinstance(skip,int), f'{skip} should be a int'
    
    sel_protein=univ.select_atoms('protein')
    xtc = f"{temp_dir}/tmp.xtc"
    pdb = f"{temp_dir}/tmp.pdb"
    
    list_of_splits=[]
    for segid in np.unique(sel_protein.segids):
        sel_segid=sel_protein.select_atoms(
            f'segid {segid}') 
        with mda.Writer(xtc, n_atoms=sel_segid.atoms.n_atoms) as W:
            for ts in univ.trajectory[::skip]:
                W.write(sel_segid)
        sel_segid.write(pdb)
        
        #Load data and calculate NMR-CS
        trj= md.load(xtc,top=pdb)
        n_frames=trj.n_frames
        print(trj.n_atoms)
        for j in tqdm(range(0,n_frames,split_size),
                      desc=f't({segid})',
                      leave=True,smoothing=1):
            list_of_splits.append(
            md.nmr.chemical_shifts_spartaplus(
                trj[j:j+split_size]))
        df = pd.concat(list_of_splits,axis=1)
        df.columns=np.arange(0,len(df.columns))       
#     os.remove(xtc)
#     os.remove(pdb)   
    return df

In [114]:
import mdtraj as md

In [112]:
df=get_sparta_plus_chemical_shifts(univ,'../data/interim/')

/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/M

1485


/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/mdtraj/nmr/shift_wrappers.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.name[results.name == "HN"] = "H"
t(PROA): 100%|██████████| 1/1 [00:03<00:00,  3.41s/it]
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: F

1485


/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/mdtraj/nmr/shift_wrappers.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.name[results.name == "HN"] = "H"
t(PROB): 100%|██████████| 1/1 [00:03<00:00,  3.63s/it]
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: F

1485


/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/mdtraj/nmr/shift_wrappers.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.name[results.name == "HN"] = "H"
t(PROC): 100%|██████████| 1/1 [00:03<00:00,  3.35s/it]
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: F

1485


/data/sperez/bin2/anaconda3/envs/lab/lib/python3.8/site-packages/mdtraj/nmr/shift_wrappers.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results.name[results.name == "HN"] = "H"
t(PROD): 100%|██████████| 1/1 [00:03<00:00,  3.56s/it]


In [115]:
df

0        1        2        3        4        5        6   \
resSeq name                                                                  
26     C     176.481  176.472  176.467  176.468  176.504  176.486  176.504   
       CA     57.292   57.282   57.275   57.278   57.325   57.302   57.325   
       CB     29.377   29.373   29.346   29.361   29.428   29.394   29.434   
       HA      4.599    4.584    4.584    4.586    4.629    4.607    4.624   
27     C     177.759  177.847  177.724  177.976  177.755  177.928  177.944   
...              ...      ...      ...      ...      ...      ...      ...   
121    CA     56.065   56.063   56.072   56.066   56.088   56.085   56.069   
       CB     30.277   30.276   30.280   30.277   30.291   30.289   30.279   
       H       8.250    8.202    8.212    8.213    8.262    8.246    8.237   
       HA      4.406    4.428    4.370    4.301    4.396    4.361    4.354   
       N     120.459  120.464  120.472  120.463  120.481  120.490  120.465   

                  7        8        9   ...       30       31       32  \
resSeq name                             ...                              
26     C     176.505  176.465  176.483  ...  176.471  176.468  176.501   
       CA     57.328   57.265   57.298  ...   57.273   57.266   57.321   
       CB     29.440   29.343   29.388  ...   29.357   29.352   29.419   
       HA      4.632    4.573    4.610  ...    4.576    4.568    4.626   
27     C     177.968  178.248  178.252  ...  178.155  178.049  177.987   
...              ...      ...      ...  ...      ...      ...      ...   
121    CA     56.077   56.069   56.074  ...   56.085   56.087   56.089   
       CB     30.284   30.262   30.262  ...   30.277   30.280   30.287   
       H       8.255    8.256    8.261  ...    8.262    8.265    8.263   
       HA      4.316    4.312    4.318  ...    4.328    4.329    4.333   
       N     120.470  120.466  120.474  ...  120.482  120.485  120.486   

                  33       34       35       36       37       38       39  
resSeq name                                                                 
26     C     176.488  176.484  176.476  176.518  176.475  176.493  176.526  
       CA     57.300   57.296   57.284   57.343   57.286   57.309   57.359  
       CB     29.392   29.378   29.365   29.444   29.382   29.407   29.476  
       HA      4.602    4.600    4.589    4.647    4.592    4.611    4.669  
27     C     178.033  177.885  178.158  178.187  178.194  177.956  178.057  
...              ...      ...      ...      ...      ...      ...      ...  
121    CA     56.080   56.083   56.079   56.058   56.093   56.076   56.039  
       CB     30.270   30.281   30.270   30.241   30.284   30.267   30.229  
       H       8.259    8.262    8.258    8.254    8.266    8.257    8.230  
       HA      4.324    4.326    4.324    4.304    4.334    4.319    4.293  
       N     120.480  120.477  120.478  120.455  120.498  120.476  120.422  

[567 rows x 40 columns]